In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from src.dnn import *
from src.data_encoders import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Post selection non-linearity

In [89]:
def non_linearity(x, weights, shots=1000):
    n = weights.shape[0]
    
    storage = qk.QuantumRegister(n, name="storage")
    clas_reg = qk.ClassicalRegister(2, name="clas_reg")
    registers = [storage, clas_reg]
    circuit = qk.QuantumCircuit(*registers)

    circuit.ry(x, storage[0])
    circuit.barrier()

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.barrier()

    for i, w in enumerate(weights):
        circuit.ry(2*w, storage[i])

    circuit.barrier()    

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.measure(storage[-2:], clas_reg)
    
    job = qk.execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)
    
    states = ["00", "01", "10", "11"]
    
    for state in states:
        if state not in counts:
            counts[state] = 0
        
    return (counts["11"])/(counts["01"] + counts["11"]), (counts["01"] + counts["11"])/shots, circuit


def gradient(x, weights):
    n = weights.shape[0]
    gradient = np.zeros(weights.shape)
    
    for i in range(n): 
        A, B, _ = non_linearity(x, weights, shots=1000)
        weights[i] +=np.pi/4
        c, d, _ = non_linearity(x, weights, shots=1000)
        weights[i] -=np.pi/2
        e, f, _ = non_linearity(x, weights, shots=1000)
        weights[i] +=np.pi/4

        C = 0.5*(c-e)
        D = 0.5*(d-f)
        
        gradient[i] = C - 2*A*D/B

    
    return gradient
    

In [90]:
np.random.seed(42)
backend = Aer.get_backend('qasm_simulator')
n = 5
weights = np.random.uniform(0, np.pi, n)
x = 0

In [93]:
for i in range(10):    
    grad = gradient(x, weights)
    y_pred, _, _ = non_linearity(x, weights, shots=10000)
    weights += -0.5*grad*(y_pred - 0.3)
    print(y_pred)

0.3059444571953179
0.29004868108230497
0.30296489833011475
0.30015941698929627
0.2979959007059895
0.29376854599406527
0.3002390982579984
0.3031132398221006
0.2972972972972973
0.29954285714285717


In [ ]:
X = np.linspace(0, np.pi, 30)
Y1 = []
Y2 = []
for x in X:
    y1, y2, _ = non_linearity(x, weights, shots=10000)
    Y1.append(y1)
    Y2.append(y2)

In [ ]:
plt.plot(X,Y1)
plt.plot(X,Y2)
plt.show()